In [1]:
import pandas as pd
import numpy as np

In [2]:
wind_data = pd.read_csv('../../data/series/sample/vortex.serie.txt', 
 header=2, delim_whitespace=True, parse_dates={'timestamp': ['YYYYMMDD', 'HHMM']}, keep_date_col=True)

In [4]:
# Define a function to calculate the circular mean
def circmean(alpha, low, high):
    """Calculate the circular mean of an array of angles (in degrees) that range from low to high"""
    alpha = np.deg2rad(alpha)
    s = np.sum(np.sin(alpha))
    c = np.sum(np.cos(alpha))
    res = np.rad2deg(np.arctan2(s, c))
    if res < 0:
        res += 360
    return res

In [6]:
# Set the timestamp column as the index
wind_data.set_index('timestamp', inplace=True)

# Calculate hourly averages of speed and direction
hourly_averages = wind_data.resample('HH').agg({'M(m/s)': 'mean', 'D(deg)': lambda x: circmean(x, 0, 360)})

# Calculate daily averages of speed and direction
daily_averages = wind_data.resample('DD').agg({'M(m/s)': 'mean', 'D(deg)': lambda x: circmean(x, 0, 360)})

# Calculate monthly averages of speed and direction
monthly_averages = wind_data.resample('MM').agg({'M(m/s)': 'mean', 'D(deg)': lambda x: circmean(x, 0, 360)})

# Calculate yearly averages of speed and direction
yearly_averages = wind_data.resample('YYYY').agg({'M(m/s)': 'mean', 'D(deg)': lambda x: circmean(x, 0, 360)})



# Print out the first few rows of each average data frame
print(hourly_averages.head())
print(daily_averages.head())
print(monthly_averages.head())
print(yearly_averages.head())

KeyError: "None of ['timestamp'] are in the columns"